# Parcel delineation using Sentinel-2
Authors:

- Kristof van Tricht (VITO)
- Jeroen Dries (VITO)
- Victor Verhaert (VITO)

Tuning:

- Kasper Bonte (VITO)
- Bart Driessen (VITO)

## Introduction
In this notebook we will be performing parcel delineation using Sentinel-2 data retrieved from- and processed on openEO. The models are generated using a U-Net and are pretrained. So in this notebook, we are dealing with the inference part of training a model. We will however also show how you can retrieve features from openEO, so that you know how the entire workflow looks like.

In [ ]:
from pathlib import Path
import openeo
from openeo import processes as eop
from shapely.geometry import box
import matplotlib.pyplot as plt
import xarray as xr
import os
from numpy import datetime_as_string

In [ ]:
## let us create a output directory

os.mkdir("results")
base_path = Path("results")

## Authentication
We first need to connect to an openEO provider. Most providers require you to register an account, and provide you with a basic amount of processing credits. In this notebook we will use the Copernicus Data Scpace Ecosystem as openEO provider.

All the known openEO providers and their services: https://hub.openeo.org/

More info on authentication: https://open-eo.github.io/openeo-python-client/auth.html

Run the authenticate_oidc() method again in case you can no longer connect. The token expires after a certain time.

In [ ]:
backend_url = "openeo.dataspace.copernicus.eu/"

eoconn = openeo.connect(backend_url)
eoconn.authenticate_oidc()

## Load collection
We start off by loading in a collection. In this case, we are loading in Sentinel-2 L2A. More information on the collections available can be displayed with eoconn.list_collections(). Use eoconn.describe_collection("SENTINEL2_L2A") for example to get the description of a specific collection.

More information on finding and loading data: https://open-eo.github.io/openeo-python-client/data_access.html

In [ ]:
bbox = [5.0, 51.2, 5.1, 51.3]
year = 2021

startdate = f"{year}-01-01"
enddate = f"{year}-09-30"

In [ ]:
s2_bands = eoconn.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=[startdate, enddate],
    spatial_extent=dict(zip(["west", "south", "east", "north"], bbox)),
    bands=["B04", "B08", "SCL"],
    max_cloud_cover=20,
)

## Select usable observations
This pipeline will use 12 NDVI tiles in total. Depending on the time range selected, it is likely that your temporal dimension contains a lot more than 12 NDVI tiles. Therefore, we apply a selection procedure that selects the tiles with the highest number of clear pixels.

In [ ]:
nb_of_timesteps = 12

In [ ]:
## We use the SCL band to create a mask where clouded pixels are set to 1
## and other pixels are set to 0
scl_band = s2_bands.band("SCL")
s2_cloudmask = ((scl_band == 8) | (scl_band == 9) | (scl_band == 3)) * 1.0

## Reduce the spacial dimension using the reducer "mean" to calculate the average cloud coverage
# TODO: replace aggregate_spacial with reduce_spacial in a future openeo client version
bbox_poly = box(*bbox)
avg_cloudmask = s2_cloudmask.aggregate_spatial(geometries=bbox_poly, reducer="mean")

## Download the result for local sorting
avg_cloudmask.download(base_path / "avg_cloudmask.nc", format="NetCDF")

In [ ]:
## Open the calculated cloudmask aggregation
avg_array = xr.open_dataset(base_path / "avg_cloudmask.nc")

## Sort the timesteps by their cloud coverage and select the best ones
best_timesteps_dt64 = (
    avg_array.squeeze("feature")
    .sortby("band_0", ascending=True)
    .coords["t"]
    .values[:nb_of_timesteps]
)

## Close the dataset
avg_array.close()

## Convert the timestep labels to iso format
best_timesteps = [
    datetime_as_string(t, unit="s", timezone="UTC") for t in best_timesteps_dt64
]

In [ ]:
## Create a condition that checks if a date is one of the best timesteps
condition = lambda x: eop.any(
    [
        eop.date_between(
            x=x, min=timestep, max=eop.date_shift(date=timestep, value=1, unit="day")
        )
        for timestep in best_timesteps
    ]
)
## Filter the bands using the condition
s2_bands_reduced = s2_bands.filter_labels(condition=condition, dimension="t")

## Calculate NDVI
The delineation will be estimated based on the NDVI. The ndvi process can be used for these calculations.

In [ ]:
ndviband = s2_bands_reduced.ndvi(red="B04", nir="B08")

> Note that the openEO Python clients generates an openEO process graph, which is then sent to one of the backends selected by the platform based on the operations that you are using. This process graph is then executed on selected platform. However, if you want to inspect intermediate results, you can, for example using ndviband.download(...) like below. Executing this line will allow you to inspect the NDVI images.

In [ ]:
ndviband.download(base_path / "ndvi.nc")

The intermediate result (here only 3 of them are ploted) of the NDVI looks as shown below:

In [ ]:
## Load your dataset
ndvi_data = xr.open_dataset(base_path / "ndvi.nc")

## Access the "var" variable
var = ndvi_data["var"]

## Select the top 3 time steps
three_steps = var["t"][-3:]

## Create a 1x3 horizontal plot for the top 3 time steps
plt.figure(figsize=(16, 10))  # Adjust the figure size as needed
for i, t in enumerate(three_steps):
    plt.subplot(1, 3, i + 1)
    data_slice = var.sel(t=t)
    dt = t.values.astype("M8[D]").astype("O")  # Convert to Python datetime object
    formatted_date = dt.strftime("%Y-%m-%d")  # Format the date
    plt.imshow(data_slice, cmap="viridis", origin="lower", vmin=0, vmax=1)
    plt.title(f"Date: {formatted_date}")
    # Hide both horizontal and vertical ticks and labels
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()  # Ensures proper spacing between subplots
plt.show()

## Close the dataset
ndvi_data.close()

![https://documentation.dataspace.copernicus.eu/APIs/openEO/openeo-community-examples/python/ParcelDelineation/Parcel%20delineation_files/figure-html/cell-13-output-1.png](https://documentation.dataspace.copernicus.eu/APIs/openEO/openeo-community-examples/python/ParcelDelineation/Parcel%20delineation_files/figure-html/cell-13-output-1.png)

## Apply a neural network
We now apply a neural network, that requires 128x128 pixel ‘chunks’ input. To avoid discontinuities between neighboring chunks, we work with an overlap of 32 pixels in all directions. The U-Net itself is trained on an input size of (x,y,1), that is, just one grayscale channel as an input, which is just one NDVI tile. However, we are using 12 NDVI images as an input (the 12 images selected in the previous step). That is because we are doing inference using 3 different models, 4 times per model. The final prediction is then the median of all predictions per pixel.

As you may know, a U-Net just like any other CNN applies a filter over an image. This can be done using apply_neighborhood, an openEO process defined here: https://openeo.org/documentation/1.0/processes.html#apply_neighborhood .

Further preprocessing of the data and the inference logic of the models is coded in an UDF. UDF’s are used to implement any custom code. It can therefore be used to quickly transfer code that you already wrote outside of openEO, or it can be used to implement features that are not present in openEO yet. It does however come at a cost of being slower than using openEO functionalities, due to optimization reasons. UDF’s are explained here: https://open-eo.github.io/openeo-python-client/udf.html

The pretrained models are converted to onnx models to allow for interoperability between different AI tools. More information on onnx and how to conveert your models from a specific framework can be found on their website. The onnxruntime package and other dependencies, used in udf_segmentation.py, have to be passed to the backend. This is done by passing a dependency archive as a job option, alongside the models.

> Important note: The memory job options have been set to make make sure that the back-end has sufficient memory to perform the inference. Larger spatial/temporal extants might require higher memory settings.

In [ ]:
dependencies_url = (
    "https://artifactory.vgt.vito.be:443/auxdata-public/openeo/onnx_dependencies.zip"
)
models_url = "https://artifactory.vgt.vito.be:443/artifactory/auxdata-public/openeo/parcelDelination/BelgiumCropMap_unet_3BandsGenerator_Models.zip"
job_options = {
    'udf-dependency-archives': [
        f"{dependencies_url}#onnx_deps",
        f"{models_url}#onnx_models"],
    'driver-memory': '500m',
    'driver-memoryOverhead': '1000m',
    'executor-memory': '1000m',
    'executor-memoryOverhead': '500m',
    'python-memory': '4000m',
 }

In [ ]:
## Apply the segmentation UDF using `apply_neighborhood`
## An overlap of 32px is used, resulting in a 128x128 pixel input
segmentationband = ndviband.apply_neighborhood(
    process=openeo.UDF.from_url("https://raw.githubusercontent.com/Open-EO/openeo-community-examples/main/python/ParcelDelineation/udf_segmentation.py"),
    size=[
        {"dimension": "x", "value": 64, "unit": "px"},
        {"dimension": "y", "value": 64, "unit": "px"},
    ],
    overlap=[
        {"dimension": "x", "value": 32, "unit": "px"},
        {"dimension": "y", "value": 32, "unit": "px"},
    ],
)

In [ ]:
segmentation_job = segmentationband.create_job(
    title="segmentation_onnx_job", out_format="NetCDF", job_options=job_options
)
segmentation_job.start_and_wait()
segmentation_job.download_result(base_path / "delineation.nc")

The result of the U-Net is a map with more clearly defined boundaries, however the result is not optimal. We will therefore post-process our U-Net result by applying non-ML filters.

In [ ]:
## Load your dataset
ds = xr.open_dataset(base_path / "delineation.nc")

## Access the "var" variable
var = ds["var"]

## Plot the data
var.plot(figsize=(10, 14), cmap="gray")  # Use a colormap that suits your data
plt.title("Parcel Delineation")
plt.show()

## Close the dataset
ds.close()

![https://documentation.dataspace.copernicus.eu/APIs/openEO/openeo-community-examples/python/ParcelDelineation/Parcel%20delineation_files/figure-html/cell-17-output-1.png](https://documentation.dataspace.copernicus.eu/APIs/openEO/openeo-community-examples/python/ParcelDelineation/Parcel%20delineation_files/figure-html/cell-17-output-1.png)

## Segmentation postprocessing
We postprocess the output from the neural network using a sobel filter and Felzenszwalb’s algorithm, which are then merged. This time, we work on larger chunks, to reduce the need for stitching the vector output.

In [ ]:
## Apply the sobel felzenszwalb UDF using `apply_neighborhood`
sobel_felzenszwalb = segmentationband.apply_neighborhood(
    process=openeo.UDF.from_url("https://raw.githubusercontent.com/Open-EO/openeo-community-examples/refs/heads/main/python/ParcelDelineation/udf_sobel_felzenszwalb.py"),
    size=[
        {"dimension": "x", "value": 256, "unit": "px"},
        {"dimension": "y", "value": 256, "unit": "px"},
    ],
    overlap=[
        {"dimension": "x", "value": 0, "unit": "px"},
        {"dimension": "y", "value": 0, "unit": "px"},
    ],
)

In [ ]:
sobel_felzenszwalb_job = sobel_felzenszwalb.create_job(
    title="sobel_felzenszwalb", out_format="NetCDF", job_options=job_options
)
sobel_felzenszwalb_job.start_and_wait()
sobel_felzenszwalb_job.download_result(base_path / "delineation_filtered.nc")

In [ ]:
## Load your dataset
ds = xr.open_dataset(base_path / "delineation_filtered.nc")

## Access the "var" variable
var = ds["var"]

## Plot the data
var.plot(figsize=(10, 14), cmap="gray")  # Use a colormap that suits your data
plt.title("Parcel Delineation - filtered")
plt.show()

## Close the dataset
ds.close()

![https://documentation.dataspace.copernicus.eu/APIs/openEO/openeo-community-examples/python/ParcelDelineation/Parcel%20delineation_files/figure-html/cell-20-output-1.png](https://documentation.dataspace.copernicus.eu/APIs/openEO/openeo-community-examples/python/ParcelDelineation/Parcel%20delineation_files/figure-html/cell-20-output-1.png)